## Downloading the Dataset

First,we need to download some prerequisite Python packages in order to run all the code below.

In [1]:
%%capture 
!pip install numpy pandas streamlit gdown pyarrow

Now we will download the datasets from Google Drive,which are in Pickle and Parquet format.

In [11]:
import os
import shutil
import gdown
import numpy as np
import pandas as pd

file_id_1 = "1m185vTdh-u7_A2ZElBvUD4SCO6oETll2"
file_id_2 = "1w41V1oWHJrBdaNJJQ4oxVBuml5CO7MQX"
downloaded_file_1 = "listings_project.pkl"
downloaded_file_2 = "calendar_project.parquet"
# Download the files from Google Drive
gdown.download(id=file_id_1, output=downloaded_file_1)
gdown.download(id=file_id_2, output=downloaded_file_2)

Downloading...
From: https://drive.google.com/uc?id=1m185vTdh-u7_A2ZElBvUD4SCO6oETll2
To: C:\Users\panos\Desktop\Personal\Extra Knowledge\Python For Data Science Course\AirBnB\listings_project.pkl
100%|█████████████████████████████████████████████████████████████████████████████| 1.42M/1.42M [00:01<00:00, 1.41MB/s]
Downloading...
From: https://drive.google.com/uc?id=1w41V1oWHJrBdaNJJQ4oxVBuml5CO7MQX
To: C:\Users\panos\Desktop\Personal\Extra Knowledge\Python For Data Science Course\AirBnB\calendar_project.parquet
100%|█████████████████████████████████████████████████████████████████████████████| 1.23M/1.23M [00:00<00:00, 1.68MB/s]


'calendar_project.parquet'

In [48]:
# Set to show all columns (instead of cascading columns in the middle)
pd.set_option("display.max_columns", None)
# And not showing numbers in scientific notation
pd.set_option("display.float_format", "{:.2f}".format)

## Preprocessing the Dataset

In [49]:
df_list = pd.read_pickle("listings_project.pkl")
df_cal = pd.read_parquet("calendar_project.parquet",engine='pyarrow')

Let's try to get an overview of the **Listings DataFrame**, called `df_list`. This should show us some details about the columns in the DataFrame, like the column names, their data types, and the number of non-null values.

In [50]:
df_list.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6165 entries, 0 to 6172
Data columns (total 34 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    6165 non-null   int64  
 1   host_acceptance_rate                  5365 non-null   float64
 2   host_is_superhost                     6165 non-null   object 
 3   host_listings_count                   6165 non-null   int64  
 4   host_total_listings_count             6165 non-null   int64  
 5   neighbourhood_cleansed                6165 non-null   object 
 6   latitude                              6165 non-null   float64
 7   longitude                             6165 non-null   float64
 8   room_type                             6165 non-null   object 
 9   accommodates                          6165 non-null   int64  
 10  bedrooms                              5859 non-null   float64
 11  beds             

We may perform this four-step process to change each of the three `discount_per_...` columns into their proper format:
1. Remove non-numeric characters, like the percent symbol, so we can perform mathematical calculations on the column
1. Change the column into a `float` data type in order to convert the data into a ratio
1. Multiply the whole column by 0.01 so we end up with a probability ratio instead of a percentage
1. Overwrite the old `discount_per_...` column with this new column

In [51]:
df_list["discount_per_5_days_booked"] = df_list["discount_per_5_days_booked"].replace('%','',regex=True).astype('float')*0.01
df_list["discount_per_10_days_booked"] = df_list["discount_per_10_days_booked"].replace('%','',regex=True).astype('float')*0.01
df_list["discount_per_30_and_more_days_booked"] = df_list["discount_per_30_and_more_days_booked"].replace('%','',regex=True).astype('float')*0.01

df_list.discount_per_5_days_booked.head(5)

0   0.05
1   0.05
2   0.07
3   0.06
4   0.09
Name: discount_per_5_days_booked, dtype: float64

Next, the columns `host_is_superhost`, `instant_bookable`, and `has_availability` are all boolean columns in the sense that their data represents true and false values, but currently are recognized as objects

In [52]:
df_list[["host_is_superhost", "instant_bookable", "has_availability"]].head(5)

,host_is_superhost,instant_bookable,has_availability
0,f,t,t
1,t,f,t
2,f,f,t
3,f,f,t
4,t,f,t


This is because the letters in these columns (**t** and **f**) are written as **strings** and not as **boolean** data types. This means we need to replace our string values with the boolean equivalent dtype.

In [53]:
df_list["host_is_superhost"] = df_list["host_is_superhost"].replace({'t' : True, 'f' : False},inplace=False).astype('bool')
df_list["instant_bookable"] = df_list["instant_bookable"].replace({'t' : True, 'f' : False},inplace=False).astype('bool')
df_list["has_availability"] = df_list["has_availability"].replace({'t' : True, 'f' : False},inplace=False).astype('bool')

df_list[["host_is_superhost", "instant_bookable", "has_availability"]].head(5)

,host_is_superhost,instant_bookable,has_availability
0,False,True,True
1,True,False,True
2,False,False,True
3,False,False,True
4,True,False,True


A closer look at the prices in the four columns `price`, `price_per_person`, `minimum_price`, and `service_cost` reveals that they all follow the same pattern:

In [54]:
df_list[["price", "price_per_person", "minimum_price", 'service_cost']].head(5)

,price,price_per_person,minimum_price,service_cost
0,$88.00,$44,$176,$4.99
1,$105.00,$52.5,$315,$4.99
2,$152.00,$38,$304,$4.99
3,$87.00,$43.5,$174,$4.99
4,$160.00,$40,$320,$4.99


All of these four columns have some special characters that we need to remove before you can change the dtype from object to float.
1. Remove dollar signs and commas
1. Convert to `float`



In [55]:
df_list["price"] = df_list["price"].str.replace('$','',regex=True).str.replace(',','',regex=True).astype('float')
df_list["price_per_person"] = df_list["price_per_person"].str.replace('$','',regex=True).str.replace(',','',regex=True).astype('float')
df_list["minimum_price"] = df_list["minimum_price"].str.replace('$','',regex=True).str.replace(',','',regex=True).astype('float')
df_list["service_cost"] = df_list["service_cost"].str.replace('$','',regex=True).str.replace(',','',regex=True).astype('float')

df_list[["price", "price_per_person", "minimum_price", 'service_cost']].head(5)

,price,price_per_person,minimum_price,service_cost
0,88.00,44.00,176.00,4.99
1,105.00,52.50,315.00,4.99
2,152.00,38.00,304.00,4.99
3,87.00,43.50,174.00,4.99
4,160.00,40.00,320.00,4.99


Once we removed dollar signs($) from price column,we must change column's name from `price` into `price_in_dollar`

We may also change column `neighbourhood_cleansed` into `neighbourhood` , just for better understanding

In [56]:
df_list = df_list.rename(columns={"price" : "price_in_dollar", "neighbourhood_cleansed" : "neighbourhood"})

Taking a closer look at the `neighbourhood` and `room_type` columns reveals that these columns are assigned an **object** dtype. We want them to be a **category** dtype.

In [57]:
df_list = df_list.astype({"neighbourhood" : "category", "room_type" : "category"})

#### Delete irrelevant columns

We need to delete some columns that are irrelevant to our current use case. Those irrelevant columns are:
* `host_listings_count`
* `host_total_listings_count`
* `availability_60`
* `availability_90`
* `availability_365`
* `number_of_reviews`
* `number_of_reviews_ltm`
* `reviews_per_month`

In [58]:
df_list = df_list.drop(
    columns=["host_listings_count", 
             "host_total_listings_count",
             "availability_60", 
             "availability_90", 
             "availability_365", 
             "number_of_reviews",
             "number_of_reviews_ltm", 
             "reviews_per_month"]
            )

## Check Our Progress 

Let's now have a look at which data types we still need to change and which columns have some null values.

In [59]:
df_list.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6165 entries, 0 to 6172
Data columns (total 26 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   id                                    6165 non-null   int64   
 1   host_acceptance_rate                  5365 non-null   float64 
 2   host_is_superhost                     6165 non-null   bool    
 3   neighbourhood                         6165 non-null   category
 4   latitude                              6165 non-null   float64 
 5   longitude                             6165 non-null   float64 
 6   room_type                             6165 non-null   category
 7   accommodates                          6165 non-null   int64   
 8   bedrooms                              5859 non-null   float64 
 9   beds                                  6082 non-null   float64 
 10  amenities                             6165 non-null   int64   
 11  pric

We can see from the output above that the columns `host_acceptance_rate`, `review_scores_rating`, `bedrooms`, `beds`, and `price_in_euros` still require some processing, as they contain missing values, and have dtypes like object or float when they need an integer data type.

In [60]:
df_list["price_in_euros"].unique()

array([None], dtype=object)

We see that column `price_in_euros` is empty so we can either `drop` it or convert values from `price_in_dollar` into `price_in_euros` according to euro rate

In [61]:
# First solution
#df_list = df_list.drop(columns=["price_in_euros"])

# Second solution
df_list["price_in_euros"] = df_list["price_in_dollar"]*0.92

In [62]:
df_list["price_in_euros"]

0       80.96
1       96.60
2      139.84
3       80.04
4      147.20
        ...  
6168   130.64
6169    87.40
6170   165.60
6171   160.08
6172    59.80
Name: price_in_euros, Length: 6165, dtype: float64

DataFrame [`info()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.info.html) revealed that some listings have no reviews and an unknown host acceptance rate. Most Airbnb users exclude such listings from their search results. To mimic this filtering approach, we filter out any rows that do not have a `review_scores_rating` and without a `host_acceptance_rate`.

In [63]:
df_list = df_list.dropna(axis=0,subset=["review_scores_rating", "host_acceptance_rate"])

After setting the right data types, you are often left with making some hard decisions and assumptions about any *partially* incomplete data in your working dataset. In this case, some `beds` and `bedrooms` have no properly assigned values. We can check this by running `df_list.info(verbose=True, show_counts=True)`, which will show that `beds` and `bedrooms` have some missing values.


Let's try and make some simple assumptions based on the `room_type` assigned to the listing. To begin with,let's inspect which room types are found in the dataset. 

In [64]:
df_list["room_type"].unique()

['Private room', 'Entire home/apt', 'Hotel room', 'Shared room']
Categories (4, object): ['Entire home/apt', 'Hotel room', 'Private room', 'Shared room']

There are four room types. Let's make the assumption that the columns `bedrooms` and `beds` are correlated with `room_type`.

Therefore, we can make the following rules:
- If we have a **"Private room"** or **"Shared room"** as `room_type`, then we believe the listing only has one bedroom. 
- If the listing has **"Hotel room"** or **"Entire home/apt"** as `room_type`, then we can divide the number of guests the listing accomodates by 2 and round up. 
- If any of these numbers are missing, then we can leave it empty.

Translating these requirements into a Python function, we get:

In [65]:
def fill_empty_bedrooms(accommodates: int, bedrooms: int, room_type: str) -> int:
    if (room_type == "Private room") or (room_type == "Shared room"):
        return 1
    elif (room_type == "Hotel room") or (room_type == "Entire home/apt"):
        return np.ceil(accommodates / 2)
    else:
        return bedrooms

### Let's time our function 

In [66]:
%%timeit -r 4 -n 100

temp_df = df_list.copy()  # Deep copy of the df, not a "view"
temp_df["rooms"] = df_list[["accommodates", "bedrooms", "room_type"]].apply(
    lambda x: fill_empty_bedrooms(x["accommodates"], x["bedrooms"], x["room_type"]),
    axis=1,
)

85.9 ms ± 19.1 ms per loop (mean ± std. dev. of 4 runs, 100 loops each)


In the case of Pandas, we are using `apply()` to semi-vectorize our function, but secretly this function just implements something that mimics a for loop. Using a `lambda` together with `apply()` allows us to access multiple columns to generate an outcome.


This approach is often good enough, but not always, especially if you are dealing with large datasets.

In [67]:
df_list["bedrooms"] = df_list[["accommodates", "bedrooms", "room_type"]].apply(lambda x: fill_empty_bedrooms(x["accommodates"], x["bedrooms"], x["room_type"]),axis=1)

Thanks to our logic and assumptions, most listings now have a proper amount of defined rooms. However, there are still a few listings without any number of rooms defined. Let's remove all rows/entries that have an empty `bedrooms`, `beds` and assign the dtype `int` instead of `float` to these two columns.

In [69]:
df_list = df_list.dropna(axis=0,subset=["bedrooms","beds"])
df_list["beds"] = df_list["beds"].astype('int')
df_list["bedrooms"] = df_list["bedrooms"].astype('int')

In [70]:
df_list.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4817 entries, 0 to 6172
Data columns (total 26 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   id                                    4817 non-null   int64   
 1   host_acceptance_rate                  4817 non-null   float64 
 2   host_is_superhost                     4817 non-null   bool    
 3   neighbourhood                         4817 non-null   category
 4   latitude                              4817 non-null   float64 
 5   longitude                             4817 non-null   float64 
 6   room_type                             4817 non-null   category
 7   accommodates                          4817 non-null   int64   
 8   bedrooms                              4817 non-null   int32   
 9   beds                                  4817 non-null   int32   
 10  amenities                             4817 non-null   int64   
 11  pric

Lastly,we can further speed-up data processing by taking the appropriate number of bytes for a given data type, especially when dealing with large datasets, and lower our memory usage significantly.

In [71]:
#Once our format of float numbers is up to 2 decimals we don't need memory for bigger than float16 numbers
df_list["price_in_dollar"] = df_list["price_in_dollar"].astype('float16')
df_list["price_per_person"] = df_list["price_per_person"].astype('float16')
df_list["minimum_price"] = df_list["minimum_price"].astype('float16')
df_list["service_cost"] = df_list["service_cost"].astype('float16')
df_list["host_acceptance_rate"] = df_list["host_acceptance_rate"].astype('float16')
df_list["review_scores_rating"] = df_list["review_scores_rating"].astype('float16')
df_list["discount_per_5_days_booked"] = df_list["discount_per_5_days_booked"].astype('float16')
df_list["discount_per_10_days_booked"] = df_list["discount_per_10_days_booked"].astype('float16')
df_list["discount_per_30_and_more_days_booked"] = df_list["discount_per_30_and_more_days_booked"].astype('float16')

# Beds,bedrooms and accomodates will never take value bigger than 32767
df_list["beds"] = df_list["beds"].astype('int16')
df_list["bedrooms"] = df_list["bedrooms"].astype('int16')
df_list["accomodates"] = df_list["accommodates"].astype('int16')

# And like this our memory usage drops significantly! (513.7 KB)
df_list.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4817 entries, 0 to 6172
Data columns (total 27 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   id                                    4817 non-null   int64   
 1   host_acceptance_rate                  4817 non-null   float16 
 2   host_is_superhost                     4817 non-null   bool    
 3   neighbourhood                         4817 non-null   category
 4   latitude                              4817 non-null   float64 
 5   longitude                             4817 non-null   float64 
 6   room_type                             4817 non-null   category
 7   accommodates                          4817 non-null   int64   
 8   bedrooms                              4817 non-null   int16   
 9   beds                                  4817 non-null   int16   
 10  amenities                             4817 non-null   int64   
 11  pric

After all the cleaning we end up using `551.3 KB` of memory instead of `1.6+ MB` we used in the beginning.

In [72]:
df_list.head(5)

,id,host_acceptance_rate,host_is_superhost,neighbourhood,latitude,longitude,room_type,accommodates,bedrooms,beds,amenities,price_in_dollar,minimum_nights,maximum_nights,has_availability,availability_30,number_of_reviews_l30d,review_scores_rating,instant_bookable,price_in_euros,price_per_person,minimum_price,discount_per_5_days_booked,discount_per_10_days_booked,discount_per_30_and_more_days_booked,service_cost,accomodates
0,23726706,0.95,False,IJburg - Zeeburgereiland,52.35,4.98,Private room,2,1,1,6,88.00,2,14,True,0,3,4.99,True,80.96,44.00,176.00,0.05,0.11,0.16,4.99,2
1,35815036,1.00,True,Noord-Oost,52.42,4.96,Entire home/apt,2,1,1,5,105.00,3,100,True,4,6,4.96,False,96.60,52.50,315.00,0.05,0.12,0.16,4.99,2
2,31553121,1.00,False,Noord-West,52.43,4.92,Entire home/apt,4,2,3,3,152.00,2,60,True,0,1,4.74,False,139.84,38.00,304.00,0.07,0.11,0.22,4.99,4
3,34745823,0.94,False,Gaasperdam - Driemond,52.30,5.01,Entire home/apt,2,1,2,8,87.00,2,1125,True,5,0,4.87,False,80.04,43.50,174.00,0.06,0.10,0.15,4.99,2
4,44586947,0.88,True,Gaasperdam - Driemond,52.31,5.03,Private room,4,1,3,4,160.00,2,31,True,9,3,5.00,False,147.20,40.00,320.00,0.09,0.22,0.20,4.99,4


In [73]:
# Calendar DataFrame
df_cal.head(5)

,listing_id,date,available,price_in_dollar,minimum_nights,maximum_nights
0,23726706,2022-06-05,False,90.00,2,1125
1,23726706,2022-06-06,False,90.00,2,1125
2,23726706,2022-06-07,False,90.00,2,1125
3,23726706,2022-06-08,False,90.00,2,1125
4,23726706,2022-06-09,False,85.00,2,1125


Now,let's assume that we are looking to stay for a minimum of 3 days. Since we are unsure when it will be booked, we would like to exclude all `listing_ids` that go below that threshold of 3 days no matter what time of year. This is because we think that bookings with a minimum stay of 3 days are more likely to have discount prices.

In [74]:
# First start by making a copy, for debugging purposes
calendar_newdf = df_cal.copy()

include_list = (
    calendar_newdf["minimum_nights"] >= 3
)

In [75]:
# Get all the listings with a minimum nights of 3+
calendar_newdf = calendar_newdf.loc[include_list]

In [76]:
calendar_newdf["five_day_dollar_price"] = calendar_newdf["price_in_dollar"] * 5

Now let's transform our newly created DataFrame into a **pivot table**, where we aggregate our rows using the `listing_id` as the index, and the columns `available` and `five_day_dollar_price` as values.

In [78]:
calendar_summarizeddf = pd.pivot_table(
    data=calendar_newdf,
    index=["listing_id"],
    values=["available", "five_day_dollar_price"],
    aggfunc=np.mean,  # The default aggregation function used
    # for merging multiple related rows of data.
)

calendar_summarizeddf.head(10)

,available,five_day_dollar_price
listing_id,,
2818,0.21,346.90
44391,0.00,1200.00
49552,0.46,1162.50
55709,0.00,818.68
55868,0.00,749.22
80635,0.00,409.25
97476,0.00,575.00
137026,0.05,1315.14
162467,0.43,761.71


Let's create a **pivot table** that states the **maximum `price_in_dollar`** for every Airbnb listing

In [79]:
temp_sum_df = pd.pivot_table(
    data = calendar_newdf,
    index = ["listing_id"],
    values = ["price_in_dollar"],
    aggfunc = np.max
)

temp_sum_df.head(10)

,price_in_dollar
listing_id,
2818,80.00
44391,240.00
49552,300.00
55709,250.00
55868,200.00
80635,89.00
97476,115.00
137026,272.00
162467,175.00


## Merge Our DataFrames

We are going to merge the pivot table that includes the `five_day_dollar_price` for each listing with `listings_df`. We have to keep in mind that we want to keep only those rows of Airbnb listing IDs that are present in both datasets.

In [81]:
final_df = pd.merge(
    df_list,
    calendar_summarizeddf,
    left_on = "id",
    right_on = "listing_id",
    how = "inner"

)
final_df.head(5)

,id,host_acceptance_rate,host_is_superhost,neighbourhood,latitude,longitude,room_type,accommodates,bedrooms,beds,amenities,price_in_dollar,minimum_nights,maximum_nights,has_availability,availability_30,number_of_reviews_l30d,review_scores_rating,instant_bookable,price_in_euros,price_per_person,minimum_price,discount_per_5_days_booked,discount_per_10_days_booked,discount_per_30_and_more_days_booked,service_cost,accomodates,available,five_day_dollar_price
0,35815036,1.00,True,Noord-Oost,52.42,4.96,Entire home/apt,2,1,1,5,105.00,3,100,True,4,6,4.96,False,96.60,52.50,315.00,0.05,0.12,0.16,4.99,2,0.66,528.89
1,19572024,1.00,False,Watergraafsmeer,52.31,4.91,Entire home/apt,6,3,6,14,279.00,3,300,True,6,3,4.69,False,256.68,46.50,837.00,0.09,0.16,0.14,4.99,6,0.82,1496.55
2,2973384,0.38,False,Watergraafsmeer,52.31,4.91,Entire home/apt,5,3,3,7,185.00,6,21,True,0,0,4.83,False,170.20,37.00,1110.00,0.06,0.12,0.18,4.99,5,0.04,941.30
3,34985473,1.00,True,Noord-Oost,52.43,5.04,Entire home/apt,5,3,4,4,135.00,6,1125,True,7,0,4.43,False,124.20,27.00,810.00,0.08,0.13,0.17,4.99,5,0.16,748.56
4,44174770,1.00,True,Watergraafsmeer,52.34,4.95,Private room,1,1,1,8,59.00,30,1125,True,0,0,4.88,True,54.28,59.00,1770.00,0.05,0.12,0.15,2.99,1,0.67,296.51


Now, let's perform a `groupby` where we look at the median values of `five_day_dollar_price` and `review_scores_accuracy` with respect to the `room_type`. 

In [82]:
final_df.groupby(by = ["room_type"])[[
    "five_day_dollar_price",
    "review_scores_rating"]].median()

,five_day_dollar_price,review_scores_rating
room_type,,
Entire home/apt,975.00,4.88
Hotel room,1110.16,4.56
Private room,710.91,4.79
Shared room,724.11,4.60


### Download the Dataset

Let's first export our final DataFrame.

In [84]:
final_df.to_csv(
    "Pandas_Airbnb_Amsterdam_listings_project.csv",
    index=True,
)